In [91]:
import pdfplumber
import re
import json

with pdfplumber.open("KKS_Equipment_key_V6.pdf") as pdf:
    equipment_types = []
    equipment_type_re = r"[A-Z] (\w+ ?)+"

    equipments = []
    equipments_re = r"([A-Z]{2} (?:[\w ,\.\-\;\(\)\*]*\n?(?![A-Z]{2}))+)"
    # This regex is monstrous
    # Needs to handle new lines that are not beginning with e.g. "AB"
    # and also needs to allow for parenthesis, dashes, stars 


    for page in pdf.pages:
        kks_table = page.extract_table()[0][1]
        found = re.search(equipment_type_re, kks_table)
        if found:
            equipment_types.append(found.group(0))
        
        found = re.findall(equipments_re, kks_table)
        if found:
            equipments.extend(found)

    
    def str2dict(data: list[str], key_len: int, sep_len=1)->dict[str]:
        """str2dict
        Convert list of strings to a dictionary
        e.g. str2dict(['A Mechanical equipment', 'G Electrical'], key_len=1) -> {"A": "Mechanical equipment", "G": "Electrical"}
        
        Parameters
        ----------
        data : list[str]
            Data to be converted
        key_len : int
            Length of string key in dictionary
        sep_len : int, optional
            Separation length between KEY and VALUE in data string, by default 1
            
        Returns
        -------
        dict[str]
            A string to string dictionary from a list of strings split according to index
        """

        return {string[:key_len]: string[key_len+sep_len:] for string in data}
        
    equipment_types = str2dict(equipment_types, key_len=1)
    equipments = str2dict(equipments, key_len=2)
    KKS_equipment = {'Type': equipment_types,
                 'Equipment': equipments}

with open('KKS_equipment.json', 'w') as fp:
    json.dump(KKS_equipment, fp)

In [93]:
with open('KKS_equipment.json', 'r') as fp:
    KKS = json.load(fp)


In [99]:
KKS["Equipment"]["CQ"]

'Quality variables (analysis, material properties), other\nthan *CD*, *CM*, *CV*'